In [5]:
# !pip install livelossplot -q
# !pip install bposd -q
# !pip install json -q

In [6]:
import os
import sys
sys.path.append(os.getcwd() + "/src")

## Benchmark random CPC codes

In [7]:
from scoring.score_dataset import run_decoder
import json
from global_params import params
from CPC import generate_random
import numpy as np
import utils

error_rates = params['eval_p_range']
n_samples = 100
best_word_succ_rates = [0.0] * len(error_rates)
best_pc = [[]] * len(error_rates)

for i, e in enumerate(error_rates):
	for _ in range(n_samples):
		pc, _, _, _ = generate_random.random_cpc()
		wsr = run_decoder(pc, 100_000, np.ones(pc.shape[-1]) * 0.001)
		if wsr > best_word_succ_rates[i]:
			best_word_succ_rates[i] = wsr
			best_pc[i] = pc

out_json = {'random_cpc': {}}
for i, r in enumerate(best_word_succ_rates):
	out_json['random_cpc'][f"error_rate_{error_rates[i]}"] = r
	out_json['random_cpc'][f"error_rate_{error_rates[i]}_pc"] = best_word_succ_rates[i]


with open(utils.get_eval_baseline_path(), "w") as outfile:
    print("WRITING", out_json)
    json.dump(out_json, outfile, cls=utils.NpEncoder)

WRITING {'random_cpc': {'error_rate_0.001': 0.998, 'error_rate_0.001_pc': 0.998, 'error_rate_0.005': 0.99804, 'error_rate_0.005_pc': 0.99804, 'error_rate_0.01': 0.99781, 'error_rate_0.01_pc': 0.99781, 'error_rate_0.015': 0.99786, 'error_rate_0.015_pc': 0.99786}}


## Benchmark AI learned codes

In [ ]:
from training import evaluate_performance
from generating import generating_model as gen_model
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scoring_model = torch.load(os.path.join(
    utils.get_best_scoring_model_path()))

generating_model = gen_model.GeneratingModel(
    device=device,
    p_skip_mutation=params['p_skip_mutation'],
    p_random_mutation=params['p_random_mutation'],
)

p_eval_range = params['eval_p_range']
evaluate_performance(scoring_model, generating_model, p_eval_range, [
                     0.001, 0.005, 0.01], "baseline_eval", False, n_tests=1_000_000)